In [1]:
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import anndata
import os
from scipy.sparse import issparse
import scvi
import scib
from scarches.models.scpoli import scPoli
import scarches as sca
from harmony import harmonize
import pyliger
from scarches.dataset.trvae.data_handling import remove_sparsity

/home/icb/korbinian.traeuble/miniconda3/envs/jupyter-experimental/lib/python3.10/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/home/icb/korbinian.traeuble/miniconda3/envs/jupyter-experimental/lib/python3.10/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (
/home/icb/korbinian.traeuble/miniconda3/envs/jupyter-experimental/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
 captum (see https://github.com/pytorch/captum).
/home/icb/korbinian.traeuble/miniconda3/envs/jupy

In this notebook the annotated datasets are concatinated, a ensembl <-> gene name mapping dictionary is created and different integration methods are implemented/executed on the dataset

# Mapping file for ensembl genes

In [ ]:
adata_final = sc.read_h5ad("../data/Plaque-datasets/small-datasets-concat.h5ad")

In [ ]:
gene_ids_fernandez = pd.read_csv("/Users/korbinian.traeuble/PhD-local/projects/main_Roche/data/Plaque-datasets/Fernandez/GSE224273/GSE224273_RAW/Sample1/genes.tsv", sep="\t", header=None)

In [ ]:
# use column "1" as index and rename column "0" to "ensembl_id"
gene_ids_fernandez = gene_ids_fernandez.set_index(1).rename(columns={0: "ensembl_id"})

In [ ]:
# Group by the index (gene name) and count the number of occurrences
grouped = gene_ids_fernandez.groupby(gene_ids_fernandez.index).size()

# Filter to get only those groups that have more than one occurrence
duplicates = grouped[grouped > 1]

# Get the actual rows from gene_ids_fernandez for these duplicates
duplicate_rows = gene_ids_fernandez.loc[duplicates.index]


In [ ]:
gene_names_adata = adata_final.var.index

In [ ]:
fixed_gene_ids = []
for gene_name in gene_names_adata:
    try:
        ensembl_ids = gene_ids_fernandez.loc[gene_name]['ensembl_id']
        # If ensembl_ids is a Series (multiple values), you can choose the first one
        ensembl_id = ensembl_ids.iloc[0] if isinstance(ensembl_ids, pd.Series) else ensembl_ids
    except KeyError:
        ensembl_id = 'not found'
    fixed_gene_ids.append(ensembl_id)



In [ ]:
fixed_gene_ids.count("not found")

In [ ]:
# get list of gene names that are not found
gene_names_not_found_fernandez = [gene_name for gene_name, ensembl_id in zip(gene_names_adata, fixed_gene_ids) if ensembl_id == "not found"]

In [ ]:
len(gene_names_not_found_fernandez)

In [ ]:
# create dataframe out of gene_names_adata and fixed_gene_ids
df_fernandez = pd.DataFrame({"gene_name": gene_names_adata, "ensembl_id": fixed_gene_ids})

In [ ]:
gene_ids_slysz = pd.read_csv("/Users/korbinian.traeuble/PhD-local/projects/main_Roche/tables/slysz_features.tsv", sep="\t", header=None)

In [ ]:
# use column "1" as index and rename column "0" to "ensembl_id"
gene_ids_slysz = gene_ids_slysz.set_index(1).rename(columns={0: "ensembl_id"}).drop(columns=[2])

In [ ]:
fixed_gene_ids_slysz = []
for gene_name in gene_names_not_found_fernandez:
    try:
        ensembl_ids = gene_ids_slysz.loc[gene_name]['ensembl_id']
        # If ensembl_ids is a Series (multiple values), you can choose the first one
        ensembl_id = ensembl_ids.iloc[0] if isinstance(ensembl_ids, pd.Series) else ensembl_ids
    except KeyError:
        ensembl_id = 'not found'
    fixed_gene_ids_slysz.append(ensembl_id)


In [ ]:
fixed_gene_ids_slysz.count("not found")

In [ ]:
# get list of gene names that are not found
gene_names_not_found_slysz = [gene_name for gene_name, ensembl_id in zip(gene_names_not_found_fernandez, fixed_gene_ids_slysz) if ensembl_id == "not found"]

In [ ]:
len(gene_names_not_found_slysz)

In [ ]:
# create dataframe out of gene_names_not_found_fernandez and fixed_gene_ids_slysz
df_slysz = pd.DataFrame({"gene_name": gene_names_not_found_fernandez, "ensembl_id": fixed_gene_ids_slysz})

In [ ]:
gene_ids_alsaigh = pd.read_csv("/Users/korbinian.traeuble/PhD-local/projects/main_Roche/tables/alsaigh_features.tsv", sep="\t", header=None)

In [ ]:
gene_ids_alsaigh= gene_ids_alsaigh.set_index(1).rename(columns={0: "ensembl_id"}).drop(columns=[2])

In [ ]:
fixed_gene_ids_alsaigh = []
for gene_name in gene_names_not_found_slysz:
    try:
        ensembl_ids = gene_ids_alsaigh.loc[gene_name]['ensembl_id']
        # If ensembl_ids is a Series (multiple values), you can choose the first one
        ensembl_id = ensembl_ids.iloc[0] if isinstance(ensembl_ids, pd.Series) else ensembl_ids
    except KeyError:
        ensembl_id = 'not found'
    fixed_gene_ids_alsaigh.append(ensembl_id)

In [ ]:
fixed_gene_ids_alsaigh.count("not found")

In [ ]:
gene_names_not_found_alsaigh = [gene_name for gene_name, ensembl_id in zip(gene_names_not_found_slysz, fixed_gene_ids_alsaigh) if ensembl_id == "not found"]

In [ ]:
# create dataframe out of gene_names_not_found_slysz and fixed_gene_ids_alsaigh
df_alsaigh = pd.DataFrame({"gene_name": gene_names_not_found_slysz, "ensembl_id": fixed_gene_ids_alsaigh})

### bring df together

In [ ]:
# delete the "not founds" from df_fernandez, df_slysz, but not df_alsaigh and merge all three
df_fernandez = df_fernandez[df_fernandez["ensembl_id"] != "not found"]
df_slysz = df_slysz[df_slysz["ensembl_id"] != "not found"]


df_merged = pd.concat([df_fernandez, df_slysz, df_alsaigh])

df_merged

In [ ]:
#count the 'not founds' in ensemble_id of df_merged
df_merged["ensembl_id"].value_counts()["not found"]

In [ ]:
# save df_merged to csv
df_merged.to_csv("../tables/gene_names_to_ensembl_ids.csv")

### check leftover genes. found out its due to seurat and scanpy make unique gene names

In [ ]:
for gene in gene_names_not_found_alsaigh:
    for dataset in adata_list:
        if gene in dataset.var_names:
            print(f"{gene} found in {dataset.obs['dataset'][0]}")
            break

In [ ]:
# get "not found" ensemble ids
not_found_ensembl_ids = df_merged[df_merged["ensembl_id"] == "not found"]["gene_name"].values

In [ ]:
for gene_name in not_found_ensembl_ids:
    gene_name_shortened = gene_name[:-2]
    if gene_name_shortened in gene_ids_alsaigh.index:
        ensembl_ids = gene_ids_alsaigh.loc[gene_name_shortened]['ensembl_id']
        ensembl_id = ensembl_ids.iloc[1] if isinstance(ensembl_ids, pd.Series) else ensembl_ids
        df_merged.loc[df_merged["gene_name"] == gene_name, "ensembl_id"] = ensembl_id
        print(f"Replaced {gene_name} with {ensembl_id}")
    else:
        print(f"{gene_name} not found")

In [ ]:
# get "not found" ensemble ids
not_found_ensembl_ids_post = df_merged[df_merged["ensembl_id"] == "not found"]["gene_name"].values

In [ ]:
not_found_ensembl_ids_post

In [ ]:
# the 5 left not found gene ids are in fact not .1 suffixed by scanpy/seurat. Looked up the gene ids manually and replaced them in the csv file

In [ ]:
manual_lookup_dict = {
    "RP11-442N24--B.1": "ENSG00000229388",
    "RP11-524D16--A.3": "ENSG00000261295",
    "RP11-59D5--B.2": "ENSG00000236345",
    "RP11-99J16--A.2": "ENSG00000244137",
    "XXyac-YX65C7-A.2": "ENSG00000226445"
}

In [ ]:
# replace the not founds in df_merged with the manual lookup dict
for gene_name, ensembl_id in manual_lookup_dict.items():
    df_merged.loc[df_merged["gene_name"] == gene_name, "ensembl_id"] = ensembl_id
    print(f"Replaced {gene_name} with {ensembl_id}")

In [ ]:
# get "not found" ensemble ids
not_found_ensembl_ids_post_post = df_merged[df_merged["ensembl_id"] == "not found"]["gene_name"].values

In [ ]:
# remove Unnamed: 0 in df_merged
df_merged = df_merged.drop(columns=["Unnamed: 0"])

In [ ]:
# save df_merged to csv
df_merged.to_csv("/Users/korbinian.traeuble/PhD-local/projects/main_Roche/tables/gene_names_to_ensembl_ids_ALLFOUND.csv", index=False)

# 1. Load and inspect the data

In [24]:
plaque_datasets_path = "/lustre/groups/epigenereg01/workspace/projects/Plaque-atlas/data/"

In [25]:
path_alsaigh = os.path.join(plaque_datasets_path, "Alsaigh/Alsaigh_annot_all_withsubclusters_withdoublets.h5ad")

In [26]:
alsaigh = sc.read_h5ad(path_alsaigh)

In [27]:
alsaigh.obs["cell_type_level1"].value_counts()

T cell          13069
Macrophage       7553
EC               2918
SMC              2665
unknown          1812
Fibroblast       1561
B cell           1558
NK               1239
Monocyte          921
Fibromyocyte      821
Mast cell         487
Plasma cell       281
doublets          100
Name: cell_type_level1, dtype: int64

In [28]:
path_wirka = os.path.join(plaque_datasets_path, "Wirka/Wirka_annot_5678.h5ad")

In [29]:
wirka = sc.read_h5ad(path_wirka)

In [30]:
wirka.obs["cell_type_level1"].value_counts()

EC              1494
Macrophage      1335
Fibroblast      1137
SMC             1134
unknown         1093
T cell           475
B cell           473
Fibromyocyte     453
Plasma cell      205
NK               101
Monocyte          78
DC                64
Mast              31
Name: cell_type_level1, dtype: int64

In [31]:
path_slysz6 = os.path.join(plaque_datasets_path, "Slysz/Slysz_fem_sample6_annot.h5ad")
path_slysz8 = os.path.join(plaque_datasets_path, "Slysz/Slysz_fem_sample8_annot.h5ad")

In [32]:
slysz6 = sc.read_h5ad(path_slysz6)
slysz8 = sc.read_h5ad(path_slysz8)

In [33]:
slysz6.obs["cell_type_level1"].value_counts()

T cell         6672
NK              892
B cell          435
Macrophage      286
Monocytes       278
Mast cell       183
unknown          64
DC               34
Plasma cell      17
Name: cell_type_level1, dtype: int64

In [34]:
slysz8.obs["cell_type_level1"].value_counts()

T cell         4148
B cell         3785
Monocyte       1138
NK              307
doublets        210
Macrophage      169
Plasma cell      94
Mast             49
unknown          21
Name: cell_type_level1, dtype: int64

In [35]:
path_pan2 = os.path.join(plaque_datasets_path, "Pan/Pan_sample2_annot.h5ad")
path_pan3 = os.path.join(plaque_datasets_path, "Pan/Pan_sample3_annot.h5ad")

In [36]:
pan2 = sc.read_h5ad(path_pan2)
pan3 = sc.read_h5ad(path_pan3)

In [37]:
pan2.obs["cell_type_level1"].value_counts()

Macrophage     714
EC             701
SMC            593
unknown        315
DC             311
T cell         299
Fibroblast     272
Mast cell      132
Monocyte       105
Plasma cell     25
B cell          19
Name: cell_type_level1, dtype: int64

In [38]:
pan3.obs["cell_type_level1"].value_counts()

SMC             708
Macrophage      513
EC              445
T cell          333
Fibromyocyte    289
DC              217
unknown         116
NK               54
Mast cell        46
Monocyte         34
B cell           12
Name: cell_type_level1, dtype: int64

In [17]:
#path_roche = os.path.join(plaque_datasets_path, "Lars_Roche2/Roche_annot.h5ad")

In [18]:
#roche = sc.read_h5ad(path_roche)

In [19]:
#roche.obs["cell_type_level1"].value_counts()

cell_type_level1
T cell          292
EC              207
Macrophage      173
Fibromyocyte    165
SMC             144
unknown         136
NK               77
Fibroblast       30
Plasma cell      24
Mast cell        18
B cell           12
Name: count, dtype: int64

In [39]:
adata_list = [alsaigh, wirka, slysz6, slysz8, pan2, pan3]# , roche]

In [21]:
adata_list

[AnnData object with n_obs × n_vars = 34985 × 22742
     obs: 'sample', 'dataset', 'symptoms', 'scDblFinder.sample', 'scDblFinder.class', 'scDblFinder.score', 'scDblFinder.weighted', 'scDblFinder.cxds_score', 'decontX_contamination', 'decontX_clusters', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'n_genes', 'size_factors', 'leiden', 'cell_type_level1', 'leiden_subclustering'
     obsm: 'X_pca', 'X_umap', 'decontX_1_UMAP', 'decontX_2_UMAP', 'decontX_3_UMAP'
     layers: 'raw_decontXcounts', 'rounded_corrected_counts', 'uncorrected_counts',
 AnnData object with n_obs × n_vars = 8073 × 20431
     obs: 'cell_type', 'sample', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'dataset', 'scDblFinder.sample', 'scDblFinder.class', 'scDblFinder.score', 'scDblFinder.weighted', 'scDblFinder.cxds_score', 'decontX_contamination', 'decontX_clusters', 'symptoms', 'size_factors', 'n_genes', 'leiden', 'cell_type_level1'
     obsm: 'X_pca', 'X_umap', '

In [40]:
adata_final = anndata.concat(adata_list, join='outer', fill_value=0.0)

In [41]:
adata_final.obs["cell_type_level1"].value_counts()

T cell          24996
Macrophage      10570
B cell           6282
EC               5558
SMC              5100
unknown          3421
Fibroblast       2970
NK               2593
Monocyte         2276
Fibromyocyte     1563
Mast cell         848
DC                626
Plasma cell       622
doublets          310
Monocytes         278
Mast               80
Name: cell_type_level1, dtype: int64

In [42]:
# rename Monocytes to Monocyte and Mast to Mast cell
adata_final.obs["cell_type_level1"] = adata_final.obs["cell_type_level1"].replace("Monocytes", "Monocyte")
adata_final.obs["cell_type_level1"] = adata_final.obs["cell_type_level1"].replace("Mast", "Mast cell")

In [43]:
#no neurons
adata_final.obs["cell_type_level1"].value_counts()

T cell          24996
Macrophage      10570
B cell           6282
EC               5558
SMC              5100
unknown          3421
Fibroblast       2970
NK               2593
Monocyte         2554
Fibromyocyte     1563
Mast cell         928
DC                626
Plasma cell       622
doublets          310
Name: cell_type_level1, dtype: int64

In [44]:
adata_final

AnnData object with n_obs × n_vars = 68093 × 31347
    obs: 'sample', 'dataset', 'symptoms', 'scDblFinder.sample', 'scDblFinder.class', 'scDblFinder.score', 'scDblFinder.weighted', 'scDblFinder.cxds_score', 'decontX_contamination', 'decontX_clusters', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'n_genes', 'size_factors', 'leiden', 'cell_type_level1', 'leiden_subclustering', 'cell_type', 'n_counts'
    obsm: 'X_pca', 'X_umap', 'decontX_1_UMAP', 'decontX_2_UMAP', 'decontX_3_UMAP', 'decontX_4_UMAP', 'decontX_5_UMAP', 'decontX_6_UMAP', 'decontX_7_UMAP', 'decontX_8_UMAP', 'decontX_1_2_3_UMAP', 'decontX_9_UMAP'
    layers: 'raw_decontXcounts', 'rounded_corrected_counts', 'uncorrected_counts'

In [45]:
adata_final.write_h5ad("small-datasets-concat-noRoche.h5ad")

# 2.1 Gene name mapping

In [46]:
ensembl_id_df = pd.read_csv("gene_names_to_ensembl_ALLFOUND_allfernandez_no6_withallslysz.csv")

In [47]:
# Create a mapping from gene names to Ensembl IDs
gene_to_ensembl = dict(zip(ensembl_id_df['gene_name'], ensembl_id_df['ensembl_id']))

In [48]:
# Map the variable names in AnnData
adata_final.var['original_gene_names'] = adata_final.var_names
adata_final.var_names = [gene_to_ensembl[gene] if gene in gene_to_ensembl else gene for gene in adata_final.var_names]

In [49]:
non_ENSG_vars = adata_final.var_names[~adata_final.var_names.str.startswith('ENSG')]
len(non_ENSG_vars)

0

In [92]:
#save .X in a new layer
adata_final.layers["log1p_scran_samplewise"] = adata.X.copy()

In [93]:
# take the rounded_corrected_counts layer as .X
adata.X = adata.layers["rounded_corrected_counts"].copy()

In [94]:
adata_final.write_h5ad("small-datasets-concat-ensembl-noRoche.h5ad")

In [56]:
# Duplicate cells (before hvg)
print(adata_final.layers["uncorrected_counts"].toarray().shape)
print(np.unique(adata_final.layers["uncorrected_counts"].toarray(), axis=0).shape)

(68093, 31347)
(68093, 31347)


# 2.2 Gene names aggregation

In [2]:
adata = sc.read_h5ad("small-datasets-concat-ensembl-noRoche.h5ad") 

In [3]:
#aggregation
# Convert the sparse matrix (if it is sparse) to a dense DataFrame
adata_df = pd.DataFrame(adata.X.toarray() if issparse(adata.X) else adata.X, 
                        index=adata.obs_names, 
                        columns=adata.var_names)

# Group by gene names and sum the counts
aggregated_data = adata_df.groupby(adata_df.columns, axis=1).sum()

# Prepare the new 'var' DataFrame, keeping the first occurrence of each gene
unique_var = adata.var.loc[~adata.var.index.duplicated(keep='first')]

# Create a new AnnData object with aggregated data
adata_agg = anndata.AnnData(X=aggregated_data, obs=adata.obs, var=unique_var.loc[aggregated_data.columns])

# 'adata_agg' now has unique gene names and aggregated counts

In [4]:
adata

AnnData object with n_obs × n_vars = 68093 × 31347
    obs: 'sample', 'dataset', 'symptoms', 'scDblFinder.sample', 'scDblFinder.class', 'scDblFinder.score', 'scDblFinder.weighted', 'scDblFinder.cxds_score', 'decontX_contamination', 'decontX_clusters', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'n_genes', 'size_factors', 'leiden', 'cell_type_level1', 'leiden_subclustering', 'cell_type', 'n_counts'
    var: 'original_gene_names'
    obsm: 'X_pca', 'X_umap', 'decontX_1_2_3_UMAP', 'decontX_1_UMAP', 'decontX_2_UMAP', 'decontX_3_UMAP', 'decontX_4_UMAP', 'decontX_5_UMAP', 'decontX_6_UMAP', 'decontX_7_UMAP', 'decontX_8_UMAP', 'decontX_9_UMAP'
    layers: 'log1p_scran_samplewise', 'raw_decontXcounts', 'rounded_corrected_counts', 'uncorrected_counts'

In [5]:
adata_agg

AnnData object with n_obs × n_vars = 68093 × 27441
    obs: 'sample', 'dataset', 'symptoms', 'scDblFinder.sample', 'scDblFinder.class', 'scDblFinder.score', 'scDblFinder.weighted', 'scDblFinder.cxds_score', 'decontX_contamination', 'decontX_clusters', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'n_genes', 'size_factors', 'leiden', 'cell_type_level1', 'leiden_subclustering', 'cell_type', 'n_counts'
    var: 'original_gene_names'

In [8]:
# MANUALLY CHECK IF SUMMATION WORKED AS INTENDED
# Extract counts for "ENSG00000033050" from the original adata
original_counts = adata[:, "ENSG00000033050"].X
if issparse(original_counts):
    original_counts = original_counts.toarray()  # Convert to dense array if sparse

# Sum these counts cellwise
summed_counts = np.sum(original_counts, axis=1)

# Extract counts for "ENSG00000033050" from adata_agg
agg_counts = adata_agg[:, "ENSG00000033050"].X
if issparse(agg_counts):
    agg_counts = agg_counts.toarray()  # Convert to dense array if sparse

# Compare the two sets of counts
comparison = np.allclose(summed_counts.flatten(), agg_counts.flatten())

# Print the result of the comparison
print(f"The counts are correctly summed: {comparison}")


The counts are correctly summed: True


In [9]:
#adata_agg.write_h5ad("small-datasets-concat-aggr-X.h5ad")

In [10]:
# Function to aggregate a layer
def aggregate_layer(layer):
    layer_df = pd.DataFrame(layer.toarray() if issparse(layer) else layer, 
                            index=adata.obs_names, 
                            columns=adata.var_names)
    return layer_df.groupby(layer_df.columns, axis=1).sum()

# Aggregate each layer and add to adata_agg
for layer_name in adata.layers.keys():
    print(f"Aggregating layer: {layer_name}")
    aggregated_layer = aggregate_layer(adata.layers[layer_name])
    
    # The first time, we need to initialize layers in adata_agg
    if not hasattr(adata_agg, 'layers'):
        adata_agg.layers = {}

    # Add the aggregated layer to adata_agg
    adata_agg.layers[layer_name] = aggregated_layer

# Now 'adata_agg' contains all the aggregated layers

Aggregating layer: log1p_scran_samplewise
Aggregating layer: raw_decontXcounts
Aggregating layer: rounded_corrected_counts
Aggregating layer: uncorrected_counts


In [11]:
def sum_gene_counts(layer, gene_name, var_names):
    # Find the index(es) of the gene
    gene_indices = np.where(var_names == gene_name)[0]

    # Sum counts across all occurrences of the gene
    gene_counts = np.sum(layer[:, gene_indices].toarray(), axis=1) if issparse(layer) else np.sum(layer[:, gene_indices], axis=1)
    return gene_counts

# Check for each layer
for layer_name in adata.layers.keys():
    print(f"Checking layer: {layer_name}")

    # Get var names for the current layer
    var_names = adata.var_names

    # Sum counts for "ENSG00000033050" in the original data
    original_counts = sum_gene_counts(adata.layers[layer_name], "ENSG00000033050", var_names)

    # Extract counts for "ENSG00000033050" from adata_agg layer
    gene_index_agg = np.where(adata_agg.var_names == "ENSG00000033050")[0]
    agg_counts = adata_agg.layers[layer_name][:, gene_index_agg]
    if issparse(agg_counts):
        agg_counts = agg_counts.toarray()

    # Compare the two sets of counts
    comparison = np.allclose(original_counts.flatten(), agg_counts.flatten())

    # Print the result of the comparison
    print(f"The counts are correctly summed in layer {layer_name}: {comparison}")



Checking layer: log1p_scran_samplewise
The counts are correctly summed in layer log1p_scran_samplewise: True
Checking layer: raw_decontXcounts
The counts are correctly summed in layer raw_decontXcounts: True
Checking layer: rounded_corrected_counts
The counts are correctly summed in layer rounded_corrected_counts: True
Checking layer: uncorrected_counts
The counts are correctly summed in layer uncorrected_counts: True


In [12]:
adata_agg

AnnData object with n_obs × n_vars = 68093 × 27441
    obs: 'sample', 'dataset', 'symptoms', 'scDblFinder.sample', 'scDblFinder.class', 'scDblFinder.score', 'scDblFinder.weighted', 'scDblFinder.cxds_score', 'decontX_contamination', 'decontX_clusters', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'n_genes', 'size_factors', 'leiden', 'cell_type_level1', 'leiden_subclustering', 'cell_type', 'n_counts'
    var: 'original_gene_names'
    layers: 'log1p_scran_samplewise', 'raw_decontXcounts', 'rounded_corrected_counts', 'uncorrected_counts'

In [13]:
import scipy.sparse as sparse

In [14]:
adata_agg.X = sparse.csr_matrix(adata_agg.X)
for layer_name in adata_agg.layers.keys():
    print("converting to sparse matrix in", layer_name)
    adata_agg.layers[layer_name] = sparse.csr_matrix(adata_agg.layers[layer_name])


converting to sparse matrix in log1p_scran_samplewise
converting to sparse matrix in raw_decontXcounts
converting to sparse matrix in rounded_corrected_counts
converting to sparse matrix in uncorrected_counts


In [15]:
adata_agg.write_h5ad("small-datasets-concat-aggr-all-sparse-noRoche.h5ad")

# 3. Normalization

In [ ]:
adata_final = sc.read_h5ad("small-datasets-concat-aggr-all-sparse-noRoche.h5ad")

In [ ]:
#Perform a clustering for scran normalization in clusters
adata_pp = adata_final.copy()
sc.pp.normalize_total(adata_pp, target_sum=1e6)
sc.pp.log1p(adata_pp)
sc.pp.pca(adata_pp, svd_solver="arpack")
sc.pp.neighbors(adata_pp, n_pcs=30)
sc.tl.leiden(adata_pp, key_added='groups', resolution=0.22)

In [ ]:
import rpy2.rinterface_lib.callbacks
import logging

from rpy2.robjects import pandas2ri
import anndata2ri

# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

# Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

In [ ]:
#Preprocess variables for scran normalization
input_groups = adata_pp.obs['groups']
data_mat = adata_final.X.T.toarray()

In [ ]:
%%R -i data_mat -i input_groups -o size_factors
library(scran)
size_factors = calculateSumFactors(data_mat, clusters=input_groups, min.mean=0.1)

In [ ]:
del adata_pp

In [ ]:
adata_final.obs['size_factors'] = size_factors

In [ ]:
#Normalize adata 
adata_final.X /= adata_final.obs['size_factors'].values[:,None]
sc.pp.log1p(adata_final)

In [ ]:
adata_final.write_h5ad("small-datasets-concat-aggr-all-sparse-scranlog1p-noRoche.h5ad")

# 4. Integrate

In [2]:
adata_final = sc.read_h5ad("small-datasets-concat-aggr-all-sparse-scranlog1p-noRoche.h5ad")

In [3]:
def prepare_adata(adata_final):
    
    #delete all layers that are not needed to reduce memory
    del adata_final.layers['log1p_scran_samplewise'] #only for small integrations
    del adata_final.layers['raw_decontXcounts']
    del adata_final.layers['uncorrected_counts']

    #rename layer to counts
    adata_final.layers['counts'] = adata_final.layers['rounded_corrected_counts']
    del adata_final.layers['rounded_corrected_counts']

    # add "_{dataset}" to the sample name to make them unique
    adata_final.obs["sample"] = adata_final.obs["sample"].astype(str) + "_" + adata_final.obs["dataset"].astype(str)
    adata_final.obs['sample'] = adata_final.obs['sample'].astype('category')   

In [4]:
def preprocess_adata(adata_final, mode, batch_key):

    # remove unknown and doublets
    print("Removing unknowns and doublets...")
    adata_final = adata_final[~adata_final.obs['cell_type_level1'].isin(['unknown', 'doublets'])].copy()

    
    if mode=="auto":
        print("Using auto mode to do hvg and pca...")
        scib.preprocessing.reduce_data(adata_final, batch_key=batch_key)
        
    elif mode=="manual":
        print("Using manual mode to do hvg and pca...")
        sc.pp.highly_variable_genes(adata_final,  n_top_genes = 2000, batch_key = batch_key)
        sc.pp.pca(adata_final, use_highly_variable=True)

    else:
        raise ValueError("Mode must be 'auto' or 'manual'")

    
    # subset only hvgs
    print("Subset for HVGs...")
    hvg = adata_final.var[adata_final.var['highly_variable']].index.tolist()
    adata_final = adata_final[:, hvg].copy()

    
    #check how many cells have zero counts for all genes
    print("Check how many cells have zero counts for all genes...")
    cellwise_sum = adata_final.X.sum(axis=1)
    num_cells_zero_counts = (cellwise_sum == 0).sum()
    
    if num_cells_zero_counts>0:
        print(num_cells_zero_counts, " cells were found with 0 counts across all genes! Removing these cells now...")
        adata_final = adata_final[cellwise_sum > 0, :]

    
    # check for dublicate gene expressions across cells after HVG selection
    print("Check for dublicate gene expressions")
    before = adata_final.layers["counts"].toarray().shape[0]
    after = np.unique(adata_final.layers["counts"].toarray(), axis=0).shape[0]
    diff = before - after

    if diff > 0:
        print(diff, " Non unique cell expression profiles found! Removing them...")
        
        counts_array = adata_final.layers["counts"].toarray()
        # Find the indices of unique rows
        _, unique_indices = np.unique(counts_array, axis=0, return_index=True)
        # Sort the indices to maintain the original order
        unique_indices_sorted = np.sort(unique_indices)
        # Filter the AnnData object to keep only unique rows
        adata_final = adata_final[unique_indices_sorted]
    else:
        "No non unique cells found."

    print("Preprocessing finished!")  

    return adata_final

In [33]:
def integrate_methods(adata, batch_key, cell_type, scgen_impl):
    
    # adata has to be filtered, hvg filtered and pca applied in obsm[X_pca]
    # log normalized data in .X and counts in .layers["counts"] 

    
    adata_final = adata

    # scGen
    print("Integrating with scGen...")

    if scgen_impl == "scib":
        print("scGen scib implementation")
        adata_after = scib.ig.scgen(adata_final, batch=batch_key, cell_type=cell_type)
        adata_final.obsm["scGen"] = adata_after.obsm["corrected_latent"].copy()
        
    elif scgen_impl == "scarches":
        print("scGen scarches implementation")

        adata_final = remove_sparsity(adata_final) # remove sparsity
        adata_final.X = adata_final.X.astype("float32")
        
        epoch = 100
        early_stopping_kwargs = {
            "early_stopping_metric": "val_loss",
            "patience": 25,
            "threshold": 0,
            "reduce_lr": True,
            "lr_patience": 13,
            "lr_factor": 0.1,
        }

        network = sca.models.scgen(adata = adata_final,
                           hidden_layer_sizes=[256,128])

        network.train(n_epochs=epoch, early_stopping_kwargs = early_stopping_kwargs, batch_size = 32)

        adata_after = network.batch_removal(adata_final, batch_key=batch_key, cell_label_key=cell_type,return_latent=True)
        adata_final.obsm["scGen"] = adata_after.obsm["corrected_latent"].copy()
    
    
    # scVI
    print("Integrating with scVI...")
    
    scvi.model.SCVI.setup_anndata(adata_final, layer="counts", batch_key=batch_key)
    vae = scvi.model.SCVI(adata_final, gene_likelihood="nb", n_layers=2, n_latent=30)
    vae.train()
    adata_final.obsm["scVI"] = vae.get_latent_representation()

    
    # scANVI
    print("Integrating with scANVI...")
    
    lvae = scvi.model.SCANVI.from_scvi_model(
        vae,
        #adata=adata_final,
        labels_key=cell_type,
        unlabeled_category="none"
    )
    lvae.train(max_epochs=40)
    adata_final.obsm["scANVI"] = lvae.get_latent_representation()

    
    # scPoli
    print("Integrating with scPoli...")
    early_stopping_kwargs = {
    "early_stopping_metric": "val_prototype_loss",
    "mode": "min",
    "threshold": 0,
    "patience": 20,
    "reduce_lr": True,
    "lr_patience": 13,
    "lr_factor": 0.1,
    }
    
    scpoli_model = scPoli(
    adata=adata_final,
    condition_keys=batch_key,
    cell_type_keys=cell_type,
    embedding_dims=5,
    recon_loss='nb',
    )
    
    scpoli_model.train(
        n_epochs=50,
        pretraining_epochs=40,
        early_stopping_kwargs=early_stopping_kwargs,
        eta=5,
    )

    scpoli_latent = scpoli_model.get_latent(adata_final)
    adata_final.obsm["scPoli"] = scpoli_latent

    
    # Harmony
    print("Integrating with Harmony...")
    adata_final.obsm["Harmony"] = harmonize(adata_final.obsm["X_pca"], adata_final.obs, batch_key=batch_key)

    
    # LIGER
    print("Integrating with LIGER...")

    batch_cats = adata_final.obs["sample"].cat.categories
    bdata = adata_final.copy()
    # Pyliger normalizes by library size with a size factor of 1
    # So here we give it the count data
    bdata.X = bdata.layers["counts"]
    # List of adata per dataset
    adata_list = [bdata[bdata.obs[batch_key] == b].copy() for b in batch_cats]
    for i, ad in enumerate(adata_list):
        ad.uns["sample_name"] = batch_cats[i]
        # Hack to make sure each method uses the same genes
        ad.uns["var_gene_idx"] = np.arange(bdata.n_vars)
    
    
    liger_data = pyliger.create_liger(adata_list, remove_missing=False, make_sparse=False)
    # Hack to make sure each method uses the same genes
    liger_data.var_genes = bdata.var_names
    pyliger.normalize(liger_data)
    pyliger.scale_not_center(liger_data)
    pyliger.optimize_ALS(liger_data, k=30)
    pyliger.quantile_norm(liger_data)
    
    
    adata_final.obsm["LIGER"] = np.zeros((adata_final.shape[0], liger_data.adata_list[0].obsm["H_norm"].shape[1]))
    for i, b in enumerate(batch_cats):
        adata_final.obsm["LIGER"][adata_final.obs[batch_key] == b] = liger_data.adata_list[i].obsm["H_norm"]

    print("Finished integrating the data")
    
    return adata_final

In [6]:
def integrate_sca(adata, batch_key, cell_type,):
    
    # adata has to be filtered, hvg filtered and pca applied in obsm[X_pca]
    # log normalized data in .X and counts in .layers["counts"] 

    
    adata_final = adata

    adata_counts = adata_final.copy()
    adata_counts.X = adata_counts.layers["counts"].copy()

    print("scGen scib implementation")
    adata_after = scib.ig.scgen(adata_final, batch=batch_key, cell_type=cell_type)
    adata_final.obsm["scGen"] = adata_after.obsm["corrected_latent"].copy()
    
    '''
    # scGen too slow
    print("Integrating with scGen...")

    adata_final = remove_sparsity(adata_final) # remove sparsity
    adata_final.X = adata_final.X.astype("float32")
    
    epoch = 100
    early_stopping_kwargs = {
        "early_stopping_metric": "val_loss",
        "patience": 25,
        "threshold": 0,
        "reduce_lr": True,
        "lr_patience": 13,
        "lr_factor": 0.1,
    }

    network = sca.models.scgen(adata = adata_final,
                       hidden_layer_sizes=[256,128])

    network.train(n_epochs=epoch, early_stopping_kwargs = early_stopping_kwargs, batch_size = 32)

    adata_after = network.batch_removal(adata_final, batch_key=batch_key, cell_label_key=cell_type,return_latent=True)
    adata_final.obsm["scGen"] = adata_after.obsm["corrected_latent"].copy()
    '''

    
    # scVI
    print("Integrating with scVI...")

    sca.models.SCVI.setup_anndata(adata_counts, batch_key=batch_key)

    vae = sca.models.SCVI(
        adata_counts,
        n_layers=2,
        encode_covariates=True,
        deeply_inject_covariates=False,
        use_layer_norm="both",
        use_batch_norm="none",
    )

    vae.train()
    adata_final.obsm["scVI"] = vae.get_latent_representation()

    
    # scANVI
    print("Integrating with scANVI...")

    scanvae = sca.models.SCANVI.from_scvi_model(vae, unlabeled_category = "none", labels_key=cell_type)
    scanvae.train(max_epochs=40)
    adata_final.obsm["scANVI"] = scanvae.get_latent_representation()

    
    
    # scPoli
    print("Integrating with scPoli...")
    early_stopping_kwargs = {
    "early_stopping_metric": "val_prototype_loss",
    "mode": "min",
    "threshold": 0,
    "patience": 20,
    "reduce_lr": True,
    "lr_patience": 13,
    "lr_factor": 0.1,
    }
    
    scpoli_model = scPoli(
    adata=adata_final,
    condition_keys=batch_key,
    cell_type_keys=cell_type,
    embedding_dims=5,
    recon_loss='mse',
    )
    
    scpoli_model.train(
        n_epochs=50,
        pretraining_epochs=40,
        early_stopping_kwargs=early_stopping_kwargs,
        eta=5,
    )

    scpoli_latent = scpoli_model.get_latent(adata_final)
    adata_final.obsm["scPoli"] = scpoli_latent


    '''
    ####################################################################################
    # scPoli counts
    print("Integrating with scPoli using raw counts...")
    early_stopping_kwargs = {
    "early_stopping_metric": "val_prototype_loss",
    "mode": "min",
    "threshold": 0,
    "patience": 20,
    "reduce_lr": True,
    "lr_patience": 13,
    "lr_factor": 0.1,
    }
    
    scpoli_model2 = scPoli(
    adata=adata_counts,
    condition_keys=batch_key,
    cell_type_keys=cell_type,
    embedding_dims=5,
    recon_loss='nb',
    )
    
    scpoli_model2.train(
        n_epochs=50,
        pretraining_epochs=40,
        early_stopping_kwargs=early_stopping_kwargs,
        eta=5,
    )

    scpoli_latent2 = scpoli_model2.get_latent(adata_counts)
    adata_final.obsm["scPoli_counts"] = scpoli_latent2
    ####################################################################################
    '''
    
    # Harmony
    print("Integrating with Harmony...")
    adata_final.obsm["Harmony"] = harmonize(adata_final.obsm["X_pca"], adata_final.obs, batch_key=batch_key)

    
    # LIGER
    print("Integrating with LIGER...")

    batch_cats = adata_final.obs["sample"].cat.categories
    bdata = adata_final.copy()
    # Pyliger normalizes by library size with a size factor of 1
    # So here we give it the count data
    bdata.X = bdata.layers["counts"]
    # List of adata per dataset
    adata_list = [bdata[bdata.obs[batch_key] == b].copy() for b in batch_cats]
    for i, ad in enumerate(adata_list):
        ad.uns["sample_name"] = batch_cats[i]
        # Hack to make sure each method uses the same genes
        ad.uns["var_gene_idx"] = np.arange(bdata.n_vars)
    
    
    liger_data = pyliger.create_liger(adata_list, remove_missing=False, make_sparse=False)
    # Hack to make sure each method uses the same genes
    liger_data.var_genes = bdata.var_names
    pyliger.normalize(liger_data)
    pyliger.scale_not_center(liger_data)
    pyliger.optimize_ALS(liger_data, k=30)
    pyliger.quantile_norm(liger_data)
    
    
    adata_final.obsm["LIGER"] = np.zeros((adata_final.shape[0], liger_data.adata_list[0].obsm["H_norm"].shape[1]))
    for i, b in enumerate(batch_cats):
        adata_final.obsm["LIGER"][adata_final.obs[batch_key] == b] = liger_data.adata_list[i].obsm["H_norm"]


    
    print("Finished integrating the data")
    
    return adata_final

In [7]:
import time
start_time = time.time()

In [4]:
prepare_adata(adata_final)

In [9]:
adata_final = preprocess_adata(adata_final, mode = "auto", batch_key="sample")

Removing unknowns and doublets...
Using auto mode to do hvg and pca...
HVG
Using 55 HVGs from full intersect set
Using 91 HVGs from n_batch-1 set
Using 166 HVGs from n_batch-2 set
Using 249 HVGs from n_batch-3 set
Using 208 HVGs from n_batch-4 set
Using 254 HVGs from n_batch-5 set
Using 315 HVGs from n_batch-6 set
Using 426 HVGs from n_batch-7 set
Using 236 HVGs from n_batch-8 set
Using 2000 HVGs
Computed 2000 highly variable genes
PCA
Nearest Neigbours
Subset for HVGs...
Check how many cells have zero counts for all genes...
17  cells were found with 0 counts across all genes! Removing these cells now...
Check for dublicate gene expressions
4  Non unique cell expression profiles found! Removing them...
Preprocessing finished!


In [ ]:
adata_final = integrate_sca(adata_final, batch_key = "sample", cell_type = "cell_type_level1")

In [11]:
end_time = time.time()
print(f"Elapsed time: {end_time - start_time} seconds")

Elapsed time: 2707.6715998649597 seconds


In [40]:
# check for dublicates in embeddings

In [12]:
for embed in ['Harmony', 'LIGER', 'X_pca', "scGen", 'scANVI', 'scPoli', 'scVI']:
    diff = adata_final.obsm[embed].shape[0] - np.unique(adata_final.obsm[embed], axis=0).shape[0]
    print(diff," for ", embed )

0  for  Harmony
0  for  LIGER
0  for  X_pca
0  for  scGen
0  for  scANVI
0  for  scPoli
0  for  scVI


In [13]:
#rename obsm X_pca to PCA in adata_final
adata_final.obsm["PCA"] = adata_final.obsm["X_pca"]
del adata_final.obsm["X_pca"]

In [14]:
adata_final.write_h5ad("scarches-noroche-mse.h5ad") # used in the 3-1_evaluation-HPC.py script